In [3]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
from bs4 import BeautifulSoup

base_url = "https://skift.com/news/"
page = 1
while True:
    # Compose the page URL (first page is just the base, subsequent pages are /page/2/, etc.)
    if page == 1:
        url = base_url
    else:
        url = f"{base_url}page/{page}/"

    response = requests.get(url)
    if response.status_code != 200:
        break  # No more pages

    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.select("article")
    if not articles:
        break  # No articles means end of pagination

    for article in articles:
        # extract info as before...
        pass

    # Proceed to next page
    # page += 1


KeyboardInterrupt: 

In [2]:
base_url = "https://skift.com/news/"
page = 1

url = f"{base_url}page/{page}/"

response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")


In [3]:
print(len(articles))

12


In [4]:
articles = soup.select("article")
print(articles)
    

[<article class="c-tease is-image-left is-size-default">
<div class="c-tease__image">
<a aria-label="What if Airbnb Builds the Killer AI Travel Search App?" class="o-aspect-ratio o-aspect-ratio--3by2 has-placeholder" href="https://skift.com/2025/07/27/what-if-airbnb-builds-the-killer-ai-travel-search-app/">
<img alt="Airbnb's app" class="" decoding="async" height="683" loading="lazy" src="https://skift.com/wp-content/uploads/2025/07/Upgrades-for-Guests-2024-Winter-Release-e1753459527633.jpg?w=1024" width="1024"> </img></a>
</div>
<div class="c-tease__content is-layout-flow">
<div class="c-tease__eyebrow has-tertiary-font-family" style="color: #f34033"><a class="" href="https://skift.com/travel-technology/">Travel Technology</a></div>
<h3 class="c-tease__title">
<a href="https://skift.com/2025/07/27/what-if-airbnb-builds-the-killer-ai-travel-search-app/">
                    What if Airbnb Builds the Killer AI Travel Search App?                </a>
</h3>
<div class="c-tease__excerpt">
 

In [16]:
articles_data = []

for article in soup.select("article"):
    label_tag = article.select_one("div.c-tease__eyebrow")
    label = label_tag.text.strip() if label_tag else None

    link_tag = article.select_one("h3.c-tease__title a")
    news_link = link_tag['href'] if link_tag else None

    author_tag = article.select_one("div.c-tease__byline a.underline")
    author = author_tag.text.strip() if author_tag else None

    time_tag = article.select_one("div.c-tease__byline time")
    news_time = time_tag.get("datetime") if time_tag else None

    articles_data.append({
        "label": label,
        "news_link": news_link,
        "author": author,
        "news_time": news_time,
        "Source": "Skift"
    })

# Preview extracted data
for item in articles_data:
    print(item)

{'label': 'Travel Technology', 'news_link': 'https://skift.com/2025/07/27/what-if-airbnb-builds-the-killer-ai-travel-search-app/', 'author': 'Sarah Kopit', 'news_time': '2025-07-27T10:39:11-04:00', 'Source': 'Skift'}
{'label': 'Airlines', 'news_link': 'https://skift.com/2025/07/26/etihad-airways-ceo-unpacks-big-bet-on-small-jets/', 'author': 'Gordon Smith', 'news_time': '2025-07-26T14:10:05-04:00', 'Source': 'Skift'}
{'label': 'Skift Meetings launch', 'news_link': 'https://meetings.skift.com/2025/07/25/5-ways-to-design-a-transformational-event-advice-from-disney/', 'author': 'Andrea Doyle', 'news_time': '2025-07-25T19:17:38-04:00', 'Source': 'Skift'}
{'label': 'Skift Meetings launch', 'news_link': 'https://meetings.skift.com/2025/07/25/human-connection-will-save-your-event-not-more-content/', 'author': 'Andrea Doyle', 'news_time': '2025-07-25T14:10:39-04:00', 'Source': 'Skift'}
{'label': 'Skift Meetings launch', 'news_link': 'https://meetings.skift.com/2025/07/25/what-your-internationa

In [19]:
import sqlite3
import hashlib
import os

DB_DIR = r"C:/Users/HP/Documents/repos/news_ingestion_data_pipeline/data"
DB_FILE = "articles.db"
DB_PATH = os.path.join(DB_DIR, DB_FILE)

def init_db():
    """Creates the articles table if it doesn't exist."""
    os.makedirs(DB_DIR, exist_ok=True)
    with sqlite3.connect(DB_PATH) as conn:
        conn.execute("""
            CREATE TABLE IF NOT EXISTS articles (
                article_id TEXT PRIMARY KEY,
                label TEXT,
                news_link TEXT,
                author TEXT,
                news_time TEXT,
                source TEXT
            )
        """)

def generate_article_id(url):
    """Generate a unique article ID by hashing the article URL."""
    return hashlib.md5(url.encode('utf-8')).hexdigest()

def insert_articles(articles):
    """
    Insert list of article dicts into the SQLite table.
    Each article dict should have keys:
    label, news_link, author, news_time, source
    """
    with sqlite3.connect(DB_PATH) as conn:
        for article in articles:
            article_id = generate_article_id(article['news_link'])
            try:
                conn.execute("""
                    INSERT OR IGNORE INTO articles 
                    (article_id, label, news_link, author, news_time, Source)
                    VALUES (?, ?, ?, ?, ?, ?)
                """, (
                    article_id,
                    article.get('label'),
                    article.get('news_link'),
                    article.get('author'),
                    article.get('news_time'),
                    article.get('Source')
                ))
            except Exception as e:
                print(f"Error inserting article {article['news_link']}: {e}")

if __name__ == "__main__":
    # Example extracted articles data
    # articles_data = [
    #     {
    #         "label": "Airlines",
    #         "news_link": "https://skift.com/2025/07/26/etihad-airways-ceo-unpacks-big-bet-on-small-jets/",
    #         "author": "Gordon Smith",
    #         "news_time": "2025-07-26T14:10:05-04:00",
    #         "source": "Skift"
    #     },
    #     # more articles ...
    # ]

    init_db()
    insert_articles(articles_data)
    print("Articles saved to SQLite database successfully!")


Articles saved to SQLite database successfully!


In [18]:
import sqlite3

DB_FILE = "articles.db"

def query_all_articles():
    # Connect to the SQLite database
    with sqlite3.connect(DB_PATH) as conn:
        cursor = conn.cursor()

        # Execute a query to select all articles
        cursor.execute("SELECT article_id, label, news_link, author, news_time, source FROM articles")

        # Fetch all rows returned by the query
        rows = cursor.fetchall()

        print(len(rows))
        # Process and display results
        for row in rows:
            article_id, label, news_link, author, news_time, source = row
            print(f"ID: {article_id}")
            print(f"Label: {label}")
            print(f"Link: {news_link}")
            print(f"Author: {author}")
            print(f"Published At: {news_time}")
            print(f"Source: {source}")
            print("-" * 40)


if __name__ == "__main__":
    query_all_articles()


12
ID: 680bdfc38e03d4dc46341415be1df193
Label: Airlines
Link: https://skift.com/2025/07/26/etihad-airways-ceo-unpacks-big-bet-on-small-jets/
Author: Gordon Smith
Published At: 2025-07-26T14:10:05-04:00
Source: Skift
----------------------------------------
ID: 112fa347e45d5fff1604a565ae5a01f2
Label: Travel Technology
Link: https://skift.com/2025/07/27/what-if-airbnb-builds-the-killer-ai-travel-search-app/
Author: Sarah Kopit
Published At: 2025-07-27T10:39:11-04:00
Source: None
----------------------------------------
ID: 38dce7c6cbc627ea4ad5e8ab0b66a866
Label: None
Link: https://meetings.skift.com/2025/07/25/5-ways-to-design-a-transformational-event-advice-from-disney/
Author: Andrea Doyle
Published At: 2025-07-25T19:17:38-04:00
Source: None
----------------------------------------
ID: 5993c1da434e0b2817b576a3201714e7
Label: None
Link: https://meetings.skift.com/2025/07/25/human-connection-will-save-your-event-not-more-content/
Author: Andrea Doyle
Published At: 2025-07-25T14:10:39-04: